#Реализация сети RNN
Теперь рассмотрим, как реализовать многослойную рекуррентную нейронную сеть. В ней будет один рекуррентный слой и один обычный полносвязный слой. Сначала создадим такую сеть с помощью Keras, а потом с помощью нашей реализации RNN слоя из предыдущего урока.

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [ ]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0


### Входной тензор
Подготовим входной тензор для тестов аналогично тому, как мы это делали в предыдущем уроке.

Размерности тензора `x`: (батч, длина цепочки, размер эмбеддинга)

In [ ]:
BATCH_SIZE = 2
SEQ_LEN = 100 # Длина последовательности
EMB_SIZE = 16 # Размер векторного представления (эмбеддинга)

x = np.random.rand(BATCH_SIZE, SEQ_LEN, EMB_SIZE).astype(np.float32)
print(x.shape)

(2, 100, 16)


### Создание простой RNN сети с помощью Keras
Создадим рекуррентную нейронную сеть с помощью `keras.Sequential`. В сети будет два слоя: один рекуррентный, один полносвязный. Рекуррентный слой будет возвращать цепочку векторов `h`, и полносвязный слой будет применяться к каждому элементу такой цепочки, так что на выходе у нас снова будет цепочка.

Дополнительный параметр  -- размерность выходных векторов `OUT_SIZE`

In [ ]:
H_SIZE = 32
OUT_SIZE = 16

rnn = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(H_SIZE, activation='relu', return_sequences=True),
    tf.keras.layers.Dense(OUT_SIZE)
])

Запускаем инференс (прямое распространение) для созданной сети и проверяем размерность выходного тензора.

Она должна была получиться `(BATCH_SIZE, SEQ_LEN, OUT_SIZE)`

In [ ]:
y = rnn(x)
print(y.shape)

(2, 100, 16)


Теперь давайте напишем свою реализацию рекуррентной нейросети (без использования `keras.Sequential` и `keras.layers.SimpleRNN`)

**[Задание 1]** Реализуйте рекуррентную нейронную сеть (класс `RNN`, наследованный от `tf.keras.Model`), эквивалентную описанной выше модели на Keras. Используйте для этого RNN ячейку (класс `RNNCell`) из предыдущего урока. 

**[Задание 2]** Сравните результаты работы класса `RNN` с оригинальной моделью на Keras так же, как мы это делали раньше (разница должна получиться равной нулю). Вам также будет необходимо скопировать веса в новую модель. В Keras модели веса находятся вот в таких переменных: `rnn.layers[i].weights[j]`

In [ ]:
# RNN ячейка из предыдущего урока
class RNNCell(tf.keras.Model):
    def __init__(self, h_size):
        super().__init__()
        self.h_size = h_size
        self.fcXH = tf.keras.layers.Dense(self.h_size)
        self.fcHH = tf.keras.layers.Dense(self.h_size, use_bias=False)
        
    def call(self, x, h):
        h = tf.nn.relu(self.fcXH(x) + self.fcHH(h))
        return h

In [ ]:
# Рекуррентная нейросеть
class RNN(tf.keras.Model):
    def __init__(self, h_size, out_size):
        super().__init__()
        self.h_size = h_size
        self.rnn_cell = RNNCell(h_size)
        self.fcHY = tf.keras.layers.Dense(out_size)

    def call(self, x_all):
        batch, length, emb_size = x.shape
        h = tf.zeros((batch, self.h_size))
        y_all = []
        
        for i in range(length):
            h = self.rnn_cell(x_all[:, i, :], h)
            y = self.fcHY(h)
            y_all.append(y)
            
        y_all = tf.transpose(tf.stack(y_all), [1, 0, 2])
        return y_all
    
rnn_my = RNN(H_SIZE, OUT_SIZE)

In [ ]:
y = rnn_my(x)
print(y.shape)

(2, 100, 16)


In [ ]:
# Перед тем, как что-то присваивать в параметры модели, нужно чтобы они создались.
# Для этого можно вызвать либо инференс с каким-то входом, либо model.build(...)

rnn_my.rnn_cell.fcXH.kernel = rnn.layers[0].weights[0]
rnn_my.rnn_cell.fcHH.kernel = rnn.layers[0].weights[1]
rnn_my.rnn_cell.fcXH.bias = rnn.layers[0].weights[2]

rnn_my.fcHY.kernel = rnn.layers[1].weights[0]
rnn_my.fcHY.bias = rnn.layers[1].weights[1]

y = rnn(x)
y_my = rnn_my(x)

print(np.max(np.abs(y.numpy() - y_my.numpy())))

1.1920929e-07
